# 增强阶段

在 RAG 流程中，检索到的文档片段往往是孤立的，缺乏足够的上下文信息。这就像盲人摸象，LLM 只能根据有限的信息进行推断，导致答案的准确性和完整性受到影响。为了解决这个问题，我们需要对检索到的上下文进行增强，为 LLM 提供更全面的信息，帮助它更好地理解文档的含义和用户的意图。上下文增强的目标是打破信息孤岛，建立信息之间的联系，从而提高 RAG 系统的整体性能。

   先说一下文档层次结构概念：
   
   这是一种在切割文档时按照特定层次进行的方法。
   层次结构包括广度和深度两个维度，例如，一个长文档可以看作是由多个章节组成，每个章节由多个段落组成，每个段落进一步分割成多个句子。在这里，章节、段落和句子之间的相互关系分别代表了不同的层次关系。

## 一、AutoMergingRetriever自动合并检索

### 自动合并检索
   自动合并检索技术将文档按照层次结构切割成不同的块，然后在检索时，将最小的结构单元（叶子节点）与查询问题进行相似度匹配。如果一个父节点上的多数叶子节点与问题匹配，则父节点文档作为上下文（context）返回给大型语言模型（LLM）。
   自动合并检索技术通过文档层次结构和自动合并的方法提高了检索的准确性和效率。
1. 加载文档建立分层节点图（父节点+子节点),会把大块文档放在父节点，并且将大块文档分割之后的小文档放在子节点
2. 查询的时候检索子节点，当查询的子节点数量足够多的时候即同一个父节点下检索子节点达到给定阈值则返回此父文档，否则不返回

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "sk-你的sk"

首先构建一个字典，包含文件的八个标准问答对，key是问题，value是期望rag的答案

In [ ]:
qna_dict = {
        "介绍下SVM算法": "是一种二分类模型，它的基本模型是定义在特征空间上的间隔最大的线性分类器，间隔最大使它有别于感知机",
        "用通俗的语言介绍下强化学习": "监督学习的特点是有一个“老师”来“监督”我们，告诉我们正确的结果是什么。在我们在小的时候，会有老师来教我们，本质上监督学习是一种知识的传递，但不能发现新的知识。对于人类整体而言，真正（甚至唯一）的知识来源是实践——也就是强化学习。比如神农尝百草，最早人类并不知道哪些草能治病，但是通\过尝试，就能学到新的知识。学习与决策者被称为智能体，与智能体交互的部分则称为环境。智能体与环境不断进行交互，具体而言，这一交互的过程可以看做是多个时刻，每一时刻，智能体根据环境的状态，依据一定的策略选择一个动作（这里的策略指的是从环境状态到智能体动作或者动作概率之间的映射），然后环境依据一定的状态转移概率转移到下一个状态，与此同时根据此时状态的好坏反馈给智能体一个奖励。智能体可以根据环境的反馈调整其策略，然后继续在环境中探索，最终学习到一个能够获得最多奖励的最优策略",
        "BN 和 LN 区别": "Batch Normalization 是对这批样本的同一维度特征做归一化， Layer Normalization 是对这单个样本的所有维度特征做归一化。区别：LN 中同层神经元输入拥有相同的均值和方差，不同的输入样本有不同的均值和方差；BN 中则针对不同神经元输入计算均值和方差，同一个 batch 中的输入拥有相同的均值和方差。所以，LN 不依赖于batch 的大小和输入 sequence 的长度，因此可以用于 batchsize 为 1 和 RNN 中 sequence 的 normalize 操作。",
        "讲讲self attention": "Self Attention 与传统的 Attention 机制非常的不同：传统的 Attention 是基于 source 端和 target 端的隐变量（hidden state）计算 Attention 的，得到的结果是源端的每个词与目标端每个词之间的依赖关系。但Self Attention 不同，它分别在 source 端和 target 端进行，仅与 source input 或者 target input 自身相关的 Self Attention，捕捉 source 端或 target 端自身的词与词之间的依赖关系；然后再把 source 端的得到的self Attention 加入到 target 端得到的 Attention 中，捕捉 source 端和 target 端词与词之间的依赖关系。因此，self Attention Attention 比传统的 Attention mechanism 效果要好，主要原因之一是，传统的Attention 机制忽略了源端或目标端句子中词与词之间的依赖关系，相对比，self Attention 可以不仅可以得到源端与目标端词与词之间的依赖关系，同时还可以有效获取源端或目标端自身词与词之间的依赖关系",
        "Bert 的预训练过程": "Bert 的预训练主要包含两个任务，MLM 和 NSP，Masked Language Model 任务可以理解为完形填空，随机 mask 每一个句子中 15%的词，用其上下文来做预测；Next Sentence Prediction 任务选择一些句子对 A 与 B，其中 50%的数据 B 是 A 的下一条句子，剩余 50%的数据 B 是语料库中随机选择的，学习其中的相关性。BERT 预训练阶段实际上是将上述两个任务结合起来，同时进行，然后将所有的 Loss 相加。",
        "GPT 与 Bert 的区别": "GPT 是单向模型，无法利用上下文信息，只能利用上文；而 BERT 是双向模型。GPT 是基于自回归模型，可以应用在 NLU 和 NLG 两大任务，而原生的 BERT 采用的基于自编码模型，只能完成 NLU 任务，无法直接应用在文本生成上面。",
        "pca 属于有监督还是无监督": "PCA 按有监督和无监督划分应该属于无监督学习，所以数据集有无 y 并不重要，只是改变样本 X 的属性(特征)维度。",
        "介绍 transformer 算法": "Transformer 本身是一个典型的 encoder-decoder 模型，Encoder 端和 Decoder 端均有 6 个 Block，Encoder 端的 Block 包括两个模块，多头 self-attention 模块以及一个前馈神经网络模块；Decoder 端的Block 包括三个模块，多头 self-attention 模块，多头 Encoder-Decoder attention 交互模块，以及一个前馈神经网络模块；需要注意：Encoder 端和 Decoder 端中的每个模块都有残差层和 Layer Normalization层。",
    }

构建评估prompt和评估函数：


意在让大模型判断生成的结果和期望的结果是否含义一致，在下面使用各种advanecd-rag方法时会经常调用， 主要使用Llama Packs，Llama Packs是对llamaindex的更加高级的封装，几行代码实现各种高级RAG，如果想看具体的流程也可以直接进入github查看源码

In [ ]:
from llama_index.core.prompts import PromptTemplate
import re
import nest_asyncio
nest_asyncio.apply()
eval_template = PromptTemplate("""
            输入:
            Question: {question}
            LLM-Answer: {llm_answer}
            Expected-Answer: {expected_answer}

            任务:
            比较LLM-Answer和Expected-Answer，确定它们是否传达类似的含义。

            输出:
            意思一致：如果LLM-Answer和Expected-Answer传达类似的含义。 
            意思不同：如果LLM-Answer和Expected-Answer传达完全不同的含义。

            示例： 
            Question：法国的首都是什么？ 
            LLM-Answer: 巴黎 
            Expected-Answer: 光之城

            输出:
            意思一致""")
    
def eval_llm_output(row, llm):
    ''' Eval llm_answer and expected_answer for the question with llm '''
    question = row['question']
    llm_answer = row['llm_answer']
    expected_answer = row['expected_answer']
    prompt_message = eval_template.format(question=question, llm_answer=llm_answer, expected_answer=expected_answer)
    answer = llm.complete(prompt_message).text
    return answer.strip()

初始化 llm

In [ ]:

from llama_index.llms.openai import OpenAI

llm = OpenAI()

载入需要查询的数据文件

In [ ]:
from llama_index.readers.file import PDFReader
from pathlib import Path

loader = PDFReader()
documents = loader.load_data(
        file=Path(f"./data/face.pdf")
    )


下载自动合并检索 pack 并构建

In [79]:
from llama_index.core.llama_pack import download_llama_pack

AutoMergingRetrieverPack = download_llama_pack(
    "AutoMergingRetrieverPack",
    "./auto_merging_retriever_pack",
    # leave the below commented out (was for testing purposes)
    # llama_hub_url="https://raw.githubusercontent.com/run-llama/llama-hub/jerry/add_llama_packs/llama_hub"
)
auto_merging_pack = AutoMergingRetrieverPack(documents)

用自动合并检索 pack 进行检索

In [80]:
import pandas as pd

answers = {'question':[], 'llm_answer': [], 'expected_answer': []}

for question in qna_dict.keys():
    expected_answer = qna_dict[question]
    # result
    response = auto_merging_pack.run(question)
    answers['question'].append(question)
    answers['llm_answer'].append(response.response)
    answers['expected_answer'].append(expected_answer)

answers_df = pd.DataFrame(answers)

> Merging 3 nodes into parent node.
> Parent node id: 0349a78c-e46c-4eb0-a93f-980ca83956cd.
> Parent node text: 第4页共46页第十七篇：2022年4月10日百度机器学习方向暑期实习面试题6道
1、介绍下SVM算法
是一种二分类模型，它的基本模型是定义在特征空间上的间隔最大的线性分类器，间隔最大使它有别于感...

> Merging 2 nodes into parent node.
> Parent node id: 5d6a3c28-7aeb-4ee0-8d51-fd30107080e0.
> Parent node text: 3、介绍svm算法
是一种二分类模型，它的基本模型是定义在特征空间上的间隔最大的线性分类器，间隔最大使它有别于感
知机。
SVM可分为三种：
线性可分SVM
当训练数据线性可分时，通过最大化硬间...

> Merging 1 nodes into parent node.
> Parent node id: 9c3f958c-7739-4b5f-98df-217ed9a30f0c.
> Parent node text: 4、传统机器学习方法了解多少
这个可以说逻辑回归模型，逻辑回归是在数据服从伯努利分布的假设下，通过极大似然的方法，运用梯度
下降法来求解参数，从而达到将数据二分类的目的。
逻辑回归的优缺点
优点：

> Merging 1 nodes into parent node.
> Parent node id: 5b4a0423-6125-4399-954a-33b99380e478.
> Parent node text: LAMB可将BERT预训练的批量大小扩展到64K，且不会造成准确率损失，76分钟就可以完成BERT的训
练。



用 `eval_llm_output` 来评估 llm-answer 和 expected-answer 是否一致

In [81]:
answers_df['rag_eval_results'] = answers_df.apply(lambda r: eval_llm_output(r, llm), axis=1)
with pd.option_context('display.max_colwidth', None):
    display(answers_df)
answers_df.to_excel('auto_merging_pack_results.xlsx', index=False)

,question,llm_answer,expected_answer,rag_eval_results
0,介绍下SVM算法,SVM算法是一种二分类模型，其基本模型是定义在特征空间上的间隔最大的线性分类器，通过最大化硬间隔可以学习得到一个线性分类器，即硬间隔SVM。当训练数据不能线性可分但是近似线性可分时，可以通过最大化软间隔学习到一个线性分类器，即软间隔SVM。此外，当训练数据线性不可分时，可以通过使用核技巧和最大化软间隔学习到一个非线性SVM。,是一种二分类模型，它的基本模型是定义在特征空间上的间隔最大的线性分类器，间隔最大使它有别于感知机,意思一致。LLM-Answer和Expected-Answer都在介绍SVM算法是一种二分类模型，其基本模型是定义在特征空间上的间隔最大的线性分类器。虽然LLM-Answer中提到了硬间隔SVM、软间隔SVM和非线性SVM的概念，但整体来看，它们传达的含义是相似的。
1,用通俗的语言介绍下强化学习,Reinforcement Learning can be explained in simple terms as a learning process where an agent learns to make decisions by receiving feedback in the form of rewards or penalties based on its actions. It is like a trial-and-error method where the agent learns through experience by interacting with its environment.,监督学习的特点是有一个“老师”来“监督”我们，告诉我们正确的结果是什么。在我们在小的时候，会有老师来教我们，本质上监督学习是一种知识的传递，但不能发现新的知识。对于人类整体而言，真正（甚至唯一）的知识来源是实践——也就是强化学习。比如神农尝百草，最早人类并不知道哪些草能治病，但是通\过尝试，就能学到新的知识。学习与决策者被称为智能体，与智能体交互的部分则称为环境。智能体与环境不断进行交互，具体而言，这一交互的过程可以看做是多个时刻，每一时刻，智能体根据环境的状态，依据一定的策略选择一个动作（这里的策略指的是从环境状态到智能体动作或者动作概率之间的映射），然后环境依据一定的状态转移概率转移到下一个状态，与此同时根据此时状态的好坏反馈给智能体一个奖励。智能体可以根据环境的反馈调整其策略，然后继续在环境中探索，最终学习到一个能够获得最多奖励的最优策略,意思不同
2,BN 和 LN 区别,LN中同层神经元输入拥有相同的均值和方差，不同的输入样本有不同的均值和方差；BN中则针对不同神经元输入计算均值和方差，同一个batch中的输入拥有相同的均值和方差。,Batch Normalization 是对这批样本的同一维度特征做归一化， Layer Normalization 是对这单个样本的所有维度特征做归一化。区别：LN 中同层神经元输入拥有相同的均值和方差，不同的输入样本有不同的均值和方差；BN 中则针对不同神经元输入计算均值和方差，同一个 batch 中的输入拥有相同的均值和方差。所以，LN 不依赖于batch 的大小和输入 sequence 的长度，因此可以用于 batchsize 为 1 和 RNN 中 sequence 的 normalize 操作。,意思一致。LLM-Answer和Expected-Answer都在解释BN和LN的区别，提到了LN是对单个样本的所有维度特征做归一化，而BN是对不同神经元输入计算均值和方差。两者都强调了LN中同层神经元输入拥有相同的均值和方差，而BN中同一个batch中的输入拥有相同的均值和方差。因此，虽然表达方式有些许不同，但传达的含义是一致的。
3,讲讲self attention,"Self attention is a mechanism that differs from traditional attention in how it operates. While traditional attention calculates dependencies between hidden states of the source and target sides, self attention focuses on capturing dependencies within the source or target input itself. By separately considering the source and target sides, self attention captures relationships between words within each side before incorporating this information into the overall attention mechanism. This approach allows self attention to effectively model dependencies within each side and between the source and target sides of the input.",Self Attention 与传统的 Attention 机制非常的不同：传统的 Attention 是基于 source 端和 target 端的隐变量（hidden state）计算 Attention 的，得到的结果是源端的每个词与目标端每个词之间的依赖关系。但Self Attention 不同，它分别在 source 端和 target 端进行，仅与 source input 或者 target input 自身相关的 Self Attention，捕捉 source 端或 target 端自身的词与词之间的依赖关系；然后再把 source 端的得到的self Attention 加入到 target 端得到的 Attention 中，捕捉 source 端和 target 端词与词之间的依赖关系。因此，self Attention Attention 比传统的 Attention mechanism 效果要好，主要原因之一是，传统的Attention 机制忽略了源端或目标端句子中词与词之间的依赖关系，相对比，self Attention 可以不仅可以得到源端与目标端词与词之间的依赖关系，同时还可以有效获取源端或目标端自身词与词之间的依赖关系,意思不同
4,Bert 的预训练过程,Bert的预训练过程主要包含两个任务，即MLM和NSP。MLM任务涉及对句子中15%的词进行随机mask，并利用上下文来预测这些被mask的词；而NSP任务则是预测两个句子是否在原始文档中是相邻的。BERT在预训练阶段同时进行这两个任务，并将它们的Loss相加。,Bert 的预训练主要包含两个任务，MLM 和 NSP，Masked Language Model 任务可以理解为完形填空，随机 mask 每一个句子中 15%的词，用其上下文来做预测；Next Sentence Prediction 任务选择一些句子对 A 与 B，其中 50%的数据 B 是 A 的下一条句子，剩余 50%的数据 B 是语料库中随机选择的，学习其中的相关性。BERT 预训练阶段实际上是将上述两个任务结合起来，同时进行，然后将所有的 Loss 相加。,意思一致。LLM-Answer和Expected-Answer都描述了Bert的预训练过程包含MLM和NSP两个任务，以及任务的具体操作方式。两者传达的含义是相似的，只是表达方式略有不同。
5,GPT 与 Bert 的区别,GPT是单向模型，只能利用上文信息，而BERT是双向模型，可以同时利用上下文信息。,GPT 是单向模型，无法利用上下文信息，只能利用上文；而 BERT 是双向模型。GPT 是基于自回归模型，可以应用在 NLU 和 NLG 两大任务，而原生的 BERT 采用的基于自编码模型，只能完成 NLU 任务，无法直接应用在文本生成上面。,意思不同
6,pca 属于有监督还是无监督,PCA belongs to unsupervised learning.,PCA 按有监督和无监督划分应该属于无监督学习，所以数据集有无 y 并不重要，只是改变样本 X 的属性(特征)维度。,意思一致。LLM-Answer和Expected-Answer传达了相同的含义，都表明PCA属于无监督学习。虽然表达方式略有不同，但传达的核心信息是一致的。
7,介绍 transformer 算法,Transformer算法是一个典型的encoder-decoder模型，其中包括Encoder端和Decoder端，每个端都由6个Block组成。Encoder端的Block包括两个模块，多头self-attention模块和前馈神经网络模块，同时每个模块都包含残差层和Layer Normalization层。在功能上，Transformer中的encoder是双向的，用于编码；而decoder是单向的，用于解码，可用于生成任务。,Transformer 本身是一个典型的 encoder-decoder 模型，Encoder 端和 Decoder 端均有 6 个 Bloc

## 二、Small-to-big Retrieval小到大块检索:
1. 输入文档分成初始父分块,将父块细分成子块,
2. 子文档和父文档建链接,并为子块建立索引。
3. 检索时中用子块,但提供对应父块给llm


下载小到大块检索 Pack 并构建

In [19]:
#pip install llama-index-embeddings-huggingface==0.2.0

from llama_index.core.llama_pack import download_llama_pack

RecursiveRetrieverSmallToBigPack = download_llama_pack(
    "RecursiveRetrieverSmallToBigPack",
    "./recursive_retriever_stb_pack",
    # leave the below commented out (was for testing purposes)
    # llama_hub_url="https://raw.githubusercontent.com/run-llama/llama-hub/jerry/add_llama_packs/llama_hub",
)

recursive_retriever_stb_pack = RecursiveRetrieverSmallToBigPack(
    documents,
)

用小到大块检索 Pack 进行检索

In [24]:
import pandas as pd

answers = {'question':[], 'llm_answer': [], 'expected_answer': []}

for question in qna_dict.keys():
    expected_answer = qna_dict[question]
    # result
    response = recursive_retriever_stb_pack.run(question)
    answers['question'].append(question)
    answers['llm_answer'].append(response.response)
    answers['expected_answer'].append(expected_answer)

answers_df = pd.DataFrame(answers)

Retrieving with query id None: 介绍下SVM算法
Retrieved node with id, entering: node-18
Retrieving with query id node-18: 介绍下SVM算法
Retrieving with query id None: 用通俗的语言介绍下强化学习
Retrieved node with id, entering: node-35
Retrieving with query id node-35: 用通俗的语言介绍下强化学习
Retrieved node with id, entering: node-0
Retrieving with query id node-0: 用通俗的语言介绍下强化学习
Retrieving with query id None: BN 和 LN 区别
Retrieved node with id, entering: node-28
Retrieving with query id node-28: BN 和 LN 区别
Retrieving with query id None: 讲讲self attention
Retrieved node with id, entering: node-7
Retrieving with query id node-7: 讲讲self attention
Retrieving with query id None: Bert 的预训练过程
Retrieved node with id, entering: node-37
Retrieving with query id node-37: Bert 的预训练过程
Retrieved node with id, entering: node-1
Retrieving with query id node-1: Bert 的预训练过程
Retrieving with query id None: GPT 与 Bert 的区别
Retrieved node with id, entering: node-8
Retrieving with query id node-8: GPT 与 Bert 的区别
Retrieving with query id None: p

用 `eval_llm_output` 来评估 llm-answer 和 expected-answer 是否一致

In [26]:
# eval llm-answer with expected-answer
answers_df['rag_eval_results'] = answers_df.apply(lambda r: eval_llm_output(r, llm), axis=1)
with pd.option_context('display.max_colwidth', None):
    display(answers_df)
answers_df.to_excel('recursive_retriever_stb_pack_results.xlsx', index=False)

,question,llm_answer,expected_answer,rag_eval_results
0,介绍下SVM算法,"The SVM algorithm's learning strategy focuses on maximizing the margin, which can be formalized as solving a convex quadratic programming problem or equivalently minimizing the regularized hinge loss function. The learning algorithm for SVM involves solving the convex quadratic programming optimization problem. SVM selects kernel functions based on the data separability - Linear kernel is primarily used for linearly separable cases with fewer parameters and faster speed, while RBF kernel is suitable for linearly inseparable cases but relies heavily on parameters, often requiring parameter tuning through techniques like cross-validation.",是一种二分类模型，它的基本模型是定义在特征空间上的间隔最大的线性分类器，间隔最大使它有别于感知机,意思不同
1,用通俗的语言介绍下强化学习,"In simple terms, reinforcement learning is a type of machine learning where an agent learns to make decisions by interacting with its environment. It works on a reward-based system, where the agent receives feedback in the form of rewards or penalties for its actions. The goal is for the agent to learn the best sequence of actions that will lead to the highest cumulative reward over time.",监督学习的特点是有一个“老师”来“监督”我们，告诉我们正确的结果是什么。在我们在小的时候，会有老师来教我们，本质上监督学习是一种知识的传递，但不能发现新的知识。对于人类整体而言，真正（甚至唯一）的知识来源是实践——也就是强化学习。比如神农尝百草，最早人类并不知道哪些草能治病，但是通\过尝试，就能学到新的知识。学习与决策者被称为智能体，与智能体交互的部分则称为环境。智能体与环境不断进行交互，具体而言，这一交互的过程可以看做是多个时刻，每一时刻，智能体根据环境的状态，依据一定的策略选择一个动作（这里的策略指的是从环境状态到智能体动作或者动作概率之间的映射），然后环境依据一定的状态转移概率转移到下一个状态，与此同时根据此时状态的好坏反馈给智能体一个奖励。智能体可以根据环境的反馈调整其策略，然后继续在环境中探索，最终学习到一个能够获得最多奖励的最优策略,意思不同。LLM-Answer介绍了强化学习的基本概念和原理，而Expected-Answer则是在比喻的方式下解释了强化学习与监督学习的区别，强调了强化学习通过实践来获取知识的特点。两者虽然都在介绍强化学习，但从不同角度和方式进行了阐述。
2,BN 和 LN 区别,BN 和 LN 区别在于，BN 是指二叉树的节点，通常表示二叉树的左节点，而 LN 则是指二维列表中的元素，通常表示二维列表中的行或列。,Batch Normalization 是对这批样本的同一维度特征做归一化， Layer Normalization 是对这单个样本的所有维度特征做归一化。区别：LN 中同层神经元输入拥有相同的均值和方差，不同的输入样本有不同的均值和方差；BN 中则针对不同神经元输入计算均值和方差，同一个 batch 中的输入拥有相同的均值和方差。所以，LN 不依赖于batch 的大小和输入 sequence 的长度，因此可以用于 batchsize 为 1 和 RNN 中 sequence 的 normalize 操作。,意思不同
3,讲讲self attention,"SelfAttention is a mechanism that differs significantly from traditional Attention. While traditional Attention calculates dependencies between hidden states of the source and target ends, SelfAttention focuses on the source or target input itself. It captures dependencies between words within the source or target input. The self-attention results from the source end are then incorporated into the attention obtained from the target end to capture dependencies between words in both the source and target ends. This approach allows SelfAttention to effectively capture dependencies not only between words in the source and target ends but also within the source or target input itself.",Self Attention 与传统的 Attention 机制非常的不同：传统的 Attention 是基于 source 端和 target 端的隐变量（hidden state）计算 Attention 的，得到的结果是源端的每个词与目标端每个词之间的依赖关系。但Self Attention 不同，它分别在 source 端和 target 端进行，仅与 source input 或者 target input 自身相关的 Self Attention，捕捉 source 端或 target 端自身的词与词之间的依赖关系；然后再把 source 端的得到的self Attention 加入到 target 端得到的 Attention 中，捕捉 source 端和 target 端词与词之间的依赖关系。因此，self Attention Attention 比传统的 Attention mechanism 效果要好，主要原因之一是，传统的Attention 机制忽略了源端或目标端句子中词与词之间的依赖关系，相对比，self Attention 可以不仅可以得到源端与目标端词与词之间的依赖关系，同时还可以有效获取源端或目标端自身词与词之间的依赖关系,意思不同。LLM-Answer和Expected-Answer虽然都在讲Self Attention，但表达方式和详细内容有所不同。LLM-Answer更加技术化和简洁，而Expected-Answer更加详细和易懂。
4,Bert 的预训练过程,"During the pre-training process of Bert, a portion of words in the sentence encoding are masked. This allows the model to predict the masked words based on the surrounding words. By artificially masking these words, the computer knows the correct values of the masked words and can assess the accuracy of the model's predictions. This process helps enhance Bert's bidirectional encoding capabilities.",Bert 的预训练主要包含两个任务，MLM 和 NSP，Masked Language Model 任务可以理解为完形填空，随机 mask 每一个句子中 15%的词，用其上下文来做预测；Next Sentence Prediction 任务选择一些句子对 A 与 B，其中 50%的数据 B 是 A 的下一条句子，剩余 50%的数据 B 是语料库中随机选择的，学习其中的相关性。BERT 预训练阶段实际上是将上述两个任

## 三、Sentence Window Retrieval句子窗口检索
1. 类似小到大块检索，先将文档解析为每个子块中的句子，
2. 对单句进行检索。并将检索到的句子连同句子窗口(当前句子的左边的句子和右边的句子)一起传递llm



下载句子窗口检索 Pack 并构建

In [14]:
from llama_index.core.llama_pack import download_llama_pack

SentenceWindowRetrieverPack = download_llama_pack(
    "SentenceWindowRetrieverPack",
    "./sentence_window_retriever_pack",
    # leave the below commented out (was for testing purposes)
    # llama_hub_url="https://raw.githubusercontent.com/run-llama/llama-hub/jerry/add_llama_packs/llama_hub",
)
sentence_window_retriever_pack = SentenceWindowRetrieverPack(
    documents,
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

D:\AAAAAAAAAAAIGC\miniconda\envs\llamaindex\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Alienware\AppData\Local\llama_index\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

用句子窗口检索 Pack 进行检索

In [17]:
import pandas as pd

answers = {'question':[], 'llm_answer': [], 'expected_answer': []}

for question in qna_dict.keys():
    expected_answer = qna_dict[question]
    # result
    response = sentence_window_retriever_pack.run(question)
    answers['question'].append(question)
    answers['llm_answer'].append(response.response)
    answers['expected_answer'].append(expected_answer)

answers_df = pd.DataFrame(answers)


用 `eval_llm_output` 来评估 llm-answer 和 expected-answer 是否一致

In [18]:
# eval llm-answer with expected-answer
answers_df['rag_eval_results'] = answers_df.apply(lambda r: eval_llm_output(r, llm), axis=1)
with pd.option_context('display.max_colwidth', None):
    display(answers_df)
answers_df.to_excel('sentence_window_retriever_pack_results.xlsx', index=False)

,question,llm_answer,expected_answer,rag_eval_results
0,介绍下SVM算法,"SVM is a binary classification model that defines the maximum margin linear classifier in the feature space, distinguishing it from the perceptron. SVM can be categorized into three types: linear separable SVM for linearly separable training data, linear SVM for data that is not linearly separable but approximately so, and nonlinear SVM for data that is not linearly separable, utilizing kernel tricks and maximizing soft margins. SVM's learning strategy focuses on maximizing the margin, which can be formalized as solving a convex quadratic programming problem or minimizing the regularized hinge loss function. The selection of the kernel function in SVM depends on the data characteristics, with Linear kernel suitable for linearly separable cases and RBF kernel for linearly inseparable cases, although parameter tuning for RBF kernel often involves cross-validation due to its dependency on parameters.",是一种二分类模型，它的基本模型是定义在特征空间上的间隔最大的线性分类器，间隔最大使它有别于感知机,意思一致。LLM-Answer和Expected-Answer都在介绍SVM算法的基本原理和特点，虽然表达方式略有不同，但传达的含义是一致的。
1,用通俗的语言介绍下强化学习,"Reinforcement learning is a type of machine learning where an agent learns to make decisions by interacting with its environment. It works on a reward system, where the agent receives positive or negative feedback based on its actions. The goal is for the agent to learn the best way to achieve a specific objective by maximizing rewards over time through trial and error.",监督学习的特点是有一个“老师”来“监督”我们，告诉我们正确的结果是什么。在我们在小的时候，会有老师来教我们，本质上监督学习是一种知识的传递，但不能发现新的知识。对于人类整体而言，真正（甚至唯一）的知识来源是实践——也就是强化学习。比如神农尝百草，最早人类并不知道哪些草能治病，但是通\过尝试，就能学到新的知识。学习与决策者被称为智能体，与智能体交互的部分则称为环境。智能体与环境不断进行交互，具体而言，这一交互的过程可以看做是多个时刻，每一时刻，智能体根据环境的状态，依据一定的策略选择一个动作（这里的策略指的是从环境状态到智能体动作或者动作概率之间的映射），然后环境依据一定的状态转移概率转移到下一个状态，与此同时根据此时状态的好坏反馈给智能体一个奖励。智能体可以根据环境的反馈调整其策略，然后继续在环境中探索，最终学习到一个能够获得最多奖励的最优策略,意思不同
2,BN 和 LN 区别,"Batch Normalization (BN) is applied to each batch of training data by normalizing based on the batch's mean and variance, while Layer Normalization (LN) normalizes across the features for each sample. LN does not depend on batch size and is used in scenarios like recurrent neural networks.",Batch Normalization 是对这批样本的同一维度特征做归一化， Layer Normalization 是对这单个样本的所有维度特征做归一化。区别：LN 中同层神经元输入拥有相同的均值和方差，不同的输入样本有不同的均值和方差；BN 中则针对不同神经元输入计算均值和方差，同一个 batch 中的输入拥有相同的均值和方差。所以，LN 不依赖于batch 的大小和输入 sequence 的长度，因此可以用于 batchsize 为 1 和 RNN 中 sequence 的 normalize 操作。,意思一致
3,讲讲self attention,"SelfAttention is a mechanism that differs significantly from traditional Attention. While traditional Attention calculates dependencies between hidden states of the source and target ends, SelfAttention operates independently on the source and target ends. It focuses solely on the self-related dependencies within the source or target input, capturing the relationships between words within the source or target end. The self-attention results from the source end are then incorporated into the attention obtained from the target end, capturing dependencies between words in both the source and target ends. This approach is more effective than traditional Attention mechanisms because it considers not only the dependencies between words in the source and target ends but also effectively captures dependencies between words within the source or target end themselves.",Self Attention 与传统的 Attention 机制非常的不同：传统的 Attention 是基于 source 端和 target 端的隐变量（hidden state）计算 Attention 的，得到的结果是源端的每个词与目标端每个词之间的依赖关系。但Self Attention 不同，它分别在 source 端和 target 端进行，仅与 source input 或者 target input 自身相关的 Self Attention，捕捉 source 端或 target 端自身的词与词之间的依赖关系；然后再把 source 端的得到的self Attention 加入到 target 端得到的 Attention 中，捕捉 source 端和 target 端词与词之间的依赖关系。因此，self Attention Attention 比传统的 Attention mechanism 效果要好，主要原因之一是，传统的Attention 机制忽略了源端或目标端句子中词与词之间的依赖关系，相对比，self Attention 可以不仅可以得到源端与目标端词与词之间的依赖关系，同时还可以有效获取源端或目标端自身词与词之间的依赖关系,意思一致。LLM-Answer和Expected-Answer都在讲述Self Attention与传统Attention机制的区别，以及Self Attention的工作原理和优势。虽然表达方式有所不同，但传达的含义是一致的。
4,Bert 的预训练过程,"Du

参考链接：https://docs.llamaindex.ai/en/v0.10.33/